<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

In [ ]:
from plan import plan; plan("avancé", "générateur")

# *avertissement* 

ce contenu peut nécessiter un ménage car il contient quelques redites par rapport à la partie 3.3 sur les itérations

### fonction génératrice et  itérateurs

# fonction génératrice

* une fonction génératrice, ou générateur, est une fonction qui retourne un itérateur
* le code d'un générateur ressemble à un code normal
* mais qui utilise le mot clé `yield` au lieu de `return`
* tout se passe donc comme si à l'exécution: 
  * on `yield` une valeur, on suspend,
  * puis plus tard à l'itération suivante  
    on restaure l'état, 

  * et on recommence
  * jusqu'à rencontrer un `return`  
    auquel cas l'itération se termine

# générateurs en pratique

In [ ]:
# un générateur
def func(n):
    for i in range(n):
        yield i**2

In [ ]:
# retourne un objet itérable
# qui donc peut etre le sujet d'un for
for i in func(10):
    print(i, end=' ') 

### fonction génératrice

* lors de l’appel d’une fonction standard
  * un espace de nommage est créé  
    pour les variables locales à la fonction

  * l’espace de nommage est détruit au `return`  
    c’est-à-dire à la sortie de la fonction

### fonction génératrice

* lors de l’appel d’une fonction générateur
  * un espace de nommage est créé  
    pour les variables locales à la fonction

  * cet espace de nommage est **conservé**  
    jusqu’à la fin de l’itération

  * ce qui peut nécessiter une pile d'exécution  
    de plusieurs stackframes en cas de `yield from`

### fonction génératrice

In [ ]:
def f():
    yield 2

In [ ]:
f()

In [ ]:
for i in f():
    print(i)

In [ ]:
# si on veut le manipuler directement
# on peut l'appeler une fois
it = f()
next(it)

In [ ]:
# mais pas la seconde dans ce cas précis
try:
    next(it)
except StopIteration as e:
    print("OOPS", e)

### fonction génératrice

In [ ]:
def func(n):
    for i in range(n):
        yield i**2

In [ ]:
for i in func(10):
    print(i, end=' ') 

* la boucle `for` appelle le générateur, 
* qui renvoie un itérateur,
* sur lequel `for` appelle `next()` comme d'habitude

### fonction génératrice

* la logique de l'itérateur
* consiste à ce que son `__next__()`
  * retourne le prochain `yield` 
  * ou à lever `StopIteration` en cas de `return`


### fonction génératrice

In [ ]:
x = func(3)
next(x)

In [ ]:
next(x)

In [ ]:
next(x)

In [ ]:
try:
    next(x)
except StopIteration as e:
    print("OOPS", e)

### fonction génératrice

In [ ]:
x = func(2)
x

In [ ]:
y = iter(x)
y

In [ ]:
z = iter(y)
z

In [ ]:
# l'itérateur d'un itérateur est lui-même, donc:
y is z

### fonction génératrice

In [ ]:
# ces trois objets sont les mêmes exactement
next(x)

In [ ]:
# ils suivent la même itération !
next(y)

In [ ]:
# du coup la troisième fois ...
try:     
    next(z)
except StopIteration as e:
    print("OOPS", e)

# `return` et `yield`

In [ ]:
def f():
    yield 1
    yield 2
    return 3
    yield 4

In [ ]:
for i in f():
    print(i)

# `return` et `yield`

* l'instruction `yield exp` dans le générateur
  * correspond à un `return exp` dans (le `next` de l') itérateur
* l'instruction `return exp` dans le générateur
  * correspond dans l'itérateur à un `raise StopIteration(exp)`
* la fin naturelle du générateur 
  * correspond dans l'itérateur à un `raise StopIteration(None)` 
* **ne pas lever** `StopIteration` dans le générateur 

# fonction générateur et itérateur

* une fonction génératrice renvoie un itérateur
* c'est donc une manière pratique de rendre un objet itérable
* que d'implémenter sa spéciale `__iter__` 
* comme une fonction génératrice

In [ ]:
# une version avec classe 
# du parcours précédent
class IterSquares:
    """les carrés de 0 à top**2"""
    def __init__(self, top):
        self.top = top
    def __iter__(self):
        i = 0
        while i <= self.top:
            yield i**2
            i += 1

In [ ]:
for i in IterSquares(3):
    print(i)

#  un itérateur pour une classe `Mots` 

* on veut implémenter une classe `Mots`
  * créée à partir d'une phrase
  * permette d'itérer sur tous les mots dans la phrase
  

##### cas 1 : `Mots` est son propre itérateur

* `Mots` a les méthodes `__iter__()` et `__next__()`
* `__iter__()` retourne `self`
* l’instance est l’itérateur, donc au maximum  
  un unique itérateur par instance

* `__next__()` implémente l’itération

In [ ]:
class Mots():
    def __init__(self, phrase):
        self.phrase = phrase.split()
        self.count = 0

    # en faisant ce choix, une instance
    # est son propre itérateur, avec la
    # limite qu'on a déjà vue plusieurs fois
    # sur les boucles imbriquées notamment
    def __iter__(self):
        return self

    def __next__(self):
        if self.count == len(self.phrase):
            raise StopIteration
        self.count = self.count + 1
        return self.phrase[self.count - 1]

In [ ]:
m = Mots("une grande phrase")
[x for x in m]

In [ ]:
# il n’y a qu’un itérateur par instance 
[x for x in m] 

###  un itérateur pour une classe `Mots` 

##### Cas 2 : l'itérateur dans une classe séparée

* `Mots` a la méthode `__iter__()` qui retourne une instance 
* d’une nouvelle classe `IterMots` qui sera un itérateur pour notre classe `Mots`
* `IterMots` a les méthodes `__iter__()`, qui retourne `self`,  
  et `__next__()` qui implémente l’itération

* chaque itération sur une instance de `Mots` crée un nouvel itérateur

In [ ]:
class Mots():
    def __init__(self, phrase):
        self.phrase = phrase.split()

    def __iter__(self):
        return IterMots(self.phrase)

class IterMots():
    def __init__(self, phrase):
        self.phrase = phrase
        self.count = 0

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.count == len(self.phrase):
            raise StopIteration
        self.count = self.count + 1
        return self.phrase[self.count - 1]

In [ ]:
m = Mots("une grande phrase")
[x for x in m]

In [ ]:
# maintenant plus de problème
[x.upper() for x in m] 

In [ ]:
# on peut itérer autant qu'on veut
[x.upper() for x in m if 'a' in x]

###  un itérateur pour une classe `Mots` 

##### Cas 3 : le plus simple

* on implémente dans `Mots` uniquement `__iter__()` 
* sous forme d’une fonction générateur
* chaque itération sur une instance crée une nouvelle fonction génératrice, donc un nouvel itérateur
* c’est la solution la plus compacte à écrire

In [ ]:
class Mots():
    def __init__(self, phrase):
        self.phrase = phrase.split()

    def __iter__(self):
        for i in self.phrase:
            yield i

In [ ]:
m = Mots("une grande phrase")
[x for x in m]

In [ ]:
[x for x in m]

In [ ]:
[x.upper() for x in m if 'a' in x]

# retour sur les générateurs

* l’utilisation des générateurs 
  * pour rendre facilement une classe itérable
  * n’est qu’une toute petite partie de leur utilité
* les générateurs sont à la base de presque tout en Python 3
  * permet d’abstraire la difficulté d’un traitement
  * très performant
  * s’utilise avec tout ce qui itère 
  * à la base des coroutines et de la programmation asynchrone

# exemple de générateur

* je veux extraire toutes les entêtes 
  * de fonctions python
  * contenues dans tous les fichiers Python d'un répertoire

In [ ]:
!grep -n 'def ' *.py /dev/null

In [ ]:
from pathlib import Path

def generator(dossier):
    for file in Path(dossier).glob("*.py"):
        try:
            with file.open() as feed:
                for lineno, line in enumerate(feed, 1):
                    if line.strip().startswith('def '):
                        yield lineno, feed.name, line.strip()[4:]
        except OSError as exc:
            print(f"oops with {file} - {type(exc)} {exc}")

In [ ]:
# on peut l'utiliser dans tout ce qui attend un itérable

# un for
for lineno, file, line in generator("."):
    print(f"{lineno}:{file}:{line}")

In [ ]:
# dans un autre generateur
def generator2():
    for (_, filename, _) in generator('.'):
        yield filename

list(generator2())


In [ ]:
# ou une expression génératrice :)

gen_exp = (filename for (_, filename, _) 
           in generator('samples'))

for x in gen_exp:
    print(x)

# exemple de générateur

## on a pas besoin de classes alors ?

* si, mais uniquement pour les cas les plus sophistiqués
  * traitement très complexe 
  * besoin d’héritage
  * etc.
* les générateurs fournissent une solution légère et élégante
  * pour abstraire des traitements itératifs

### `yield from`, ou comment factoriser les générateurs

In [ ]:
def sum_numbers(num):
    """
    retourne la somme de tous les chiffres d'un nombre
    récursivement jusqu'à obtenir un chiffre entre 0 et 9
    """
    _sum = 0
    for n in str(num):
        _sum = _sum + int(n)
    if _sum > 10:
        return sum_numbers(_sum)
    else:
        return _sum


In [ ]:
sum_numbers(11236786578)

In [ ]:
# comment factoriser du code qui contient un yield ?

def gen(size):
    # if size is even: make it odd
    if size % 2 == 0:
        size = size - 1
        for i in range(size):         # le même 
            if sum_numbers(i) == 7:   # fragment 
                yield i               # de code
    else:
        for i in range(size):         # utilisé
            if sum_numbers(i) == 7:   # deux fois
                yield i               # mais avec un yield

In [ ]:
# première solution, qui marche mais peu élégante

# le code factorisé
def gen_sum_is_7(size):
    for i in range(size):
        if sum_numbers(i) == 7:
            yield i

def gen(size):
    # if size is even: make it odd
    if size % 2 == 0:        
        size = size - 1      
        # on ne peut utiliser le code factorisé
        # que via un for, du coup c'est lourd
        for i in gen_sum_is_7(size):
            yield i
    else:
        # ditto
        for i in gen_sum_is_7(size):
            yield i

In [ ]:
# deuxième solution: yield from

# le code factorisé (inchangé p/r solution 1)
def gen_sum_is_7(size):
    for i in range(size):
        if sum_numbers(i) == 7:
            yield i

def gen(size):
    # if size is even: make it odd
    if size % 2 == 0:        
        size = size - 1      
        # on ne peut utiliser le code factorisé
        # que via un for, du coup c'est lourd
        yield from gen_sum_is_7(size)
    else:
        # ditto
        yield from gen_sum_is_7(size)

# `yield from`

* beaucoup plus puissant que cela
* on en reparlera avec les coroutines et `asyncio`
  * c'est de la délégation

# chaîne de traitements

* un usage classique des générateurs est de les chaîner
  * pour faire une chaîne de traitements
* l’idée est de cascader plusieurs traitements
  * sans jamais générer une grande structure de données temporaire

In [ ]:
# ces trois fonctions utilisent yield et sont donc des générateurs 
def cat_on_file(filename):
    print("ouverture de {}".format(filename))
    with open(filename, 'r') as f:
        for line in f:
            yield line.strip()

# ces deux fonctions attendent un itérable 'lines'
# et peuvent donc travailler 
# sur (le résultat d')un générateur
def remove_comments(lines):
    for line in lines:
        if not line.startswith('#'):
            yield line


def get_func_headers(lines):
    for line in lines:
        if line.startswith('def') and line.endswith(':'):
            yield line

In [ ]:
# all_lines est l'itérateur rendu par cat_on_files
all_lines = cat_on_file('samples/closures.py')

In [ ]:
# on peut donc appeler remove_comments dessus
all_lines_no_comment = remove_comments(all_lines)

In [ ]:
# et à nouveau
all_functions = get_func_headers(all_lines_no_comment)

* remarquez bien qu'à ce stade on n'a toujours pas ouvert le fichier !

In [ ]:
for i in all_functions:
    print(i)

In [ ]:
# si on recommence il ne se passe plus rien
for i in all_functions:
    print(i)